In [2]:
import matplotlib.pyplot as plt
# external imports
import transformers
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import torch
import torchvision
import time 
import numpy as np
from PIL import Image
from PIL import ImageFile
import requests
import datasets
from datasets import load_dataset
from torchvision import datasets, transforms
from tqdm import tqdm
import os
import cv2

In [5]:
torch.cuda.is_available()

True

# GoPro with DepthAnything


## Large model


In [2]:
# Set this to True to avoid errors with truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\imgs"
image_files = os.listdir(image_folder)

results_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\processed\\depth_maps_large\\"
pretty_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\processed\\pretty_large\\"

image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf")

# Move model and image processor to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for filename in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path)

    already_processed = False
    # skip if already done
    for root, dirs, files in os.walk(pretty_folder):
        for file in files:
            if filename==file:
               already_processed = True

    if already_processed:
       continue

    
    # Prepare image for the model
    inputs = image_processor(images=image, return_tensors="pt")
    # Move to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}
 
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth
    
    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=image.size[::-1],
        mode="bicubic",
        align_corners=False,
    )
    
    # save the prediction
    output = prediction.squeeze().cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype("uint8")
    depth = Image.fromarray(formatted)
    depth.save(os.path.join(results_folder, filename))
    
    ground_name = filename[:-5] + "_SeaErra_abs_depth.tif"
    ground_truth_path = os.path.join(depth_folder, ground_name)
    gt_image = Image.open(ground_truth_path)
    gt = np.array(gt_image)
    formatted_gt = (gt * 255 / np.max(output)).astype("uint8")
    ground_truth = Image.fromarray(formatted_gt)
    
    # save the prediction in a pretty way
    fig, ax = plt.subplots(1, 3, dpi=400)

    ax[0].imshow(image)
    ax[0].axis('off')
    ax[0].set_title('FLSea')
    ax[1].imshow(depth, cmap= 'plasma')
    ax[1].axis('off')
    ax[1].set_title('DepthAnything')
    ax[2].imshow(ground_truth, cmap= 'plasma_r')
    ax[2].axis('off')
    ax[2].set_title('Ground Truth')
    
    fig.savefig(os.path.join(pretty_folder, filename))
    plt.close(fig)

Processing images: 100%|█████████████████████████████████████████████████████████████| 176/176 [14:05<00:00,  4.80s/it]


## Small model

In [16]:
# Set this to True to avoid errors with truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\imgs"
image_files = os.listdir(image_folder)
# 16233165812779584

depth_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\depth"
# depth_files = os.listdir(depth_folder)

# 16233165812779584_SeaErra_abs_depth

results_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\processed\\depth_maps_small\\"
pretty_folder = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\processed\\pretty_small\\"

image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf")

# Move model and image processor to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for filename in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path)

    already_processed = False
    # skip if already done
    ##for root, dirs, files in os.walk(pretty_folder):
    #    for file in files:
    #       if filename==file:
        #       already_processed = True
#
   # if already_processed:
    #   continue

    
    # Prepare image for the model
    inputs = image_processor(images=image, return_tensors="pt")
    # Move to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth
    
    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=image.size[::-1],
        mode="bicubic",
        align_corners=False,
    )
    
    # save the prediction
    output = prediction.squeeze().cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype("uint8")
    depth = Image.fromarray(formatted)
    depth.save(os.path.join(results_folder, filename))

    ground_name = filename[:-5] + "_SeaErra_abs_depth.tif"
    ground_truth_path = os.path.join(depth_folder, ground_name)
    gt_image = Image.open(ground_truth_path)
    gt = np.array(gt_image)
    formatted_gt = (gt * 255 / np.max(output)).astype("uint8")
    ground_truth = Image.fromarray(formatted_gt)
    
    # save the prediction in a pretty way
    fig, ax = plt.subplots(1, 3, dpi=400)

    ax[0].imshow(image)
    ax[0].axis('off')
    ax[0].set_title('FLSea')
    ax[1].imshow(depth, cmap= 'plasma')
    ax[1].axis('off')
    ax[1].set_title('DepthAnything')
    ax[2].imshow(ground_truth, cmap= 'plasma_r')
    ax[2].axis('off')
    ax[2].set_title('Ground Truth')
    
    fig.savefig(os.path.join(pretty_folder, filename))
    plt.close(fig)

Processing images: 100%|███████████████████████████████████████████████████████████| 1012/1012 [07:41<00:00,  2.19it/s]
